## Here we get a better understanding of what is happening in `script.py`

In [9]:
!pip install transformers


[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [10]:
import torch
import numpy as np
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertTokenizer, DistilBertModel
from tqdm import tqdm
import argparse
import os
import pandas as pd

In [11]:
s3_path = 's3://hugging-face-text-multiclass-text-classification-bucket/training_data/newsCorpora.csv'
df = pd.read_csv(s3_path, sep='\t', names=['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP'])

# remember we want to classify the title based on the category

df = df[['TITLE', 'CATEGORY']]


my_dict = {
    'e':'Entertainment',
    'b':'Business',
    't':'Science',
    'm':'Health'
}

# Change the 'CATEGORY' columns' cryptic symbols to accord with more readable dictionary. x in the function represents symbols
def update_cat(x):
    return my_dict[x]

# Update the CATEGORY column
df['CATEGORY'] = df['CATEGORY'].apply(lambda x: update_cat(x))

print(df)

                                                    TITLE  CATEGORY
0       Fed official says weak data caused by weather,...  Business
1       Fed's Charles Plosser sees high bar for change...  Business
2       US open: Stocks fall after Fed official hints ...  Business
3       Fed risks falling 'behind the curve', Charles ...  Business
4       Fed's Plosser: Nasty Weather Has Curbed Job Gr...  Business
...                                                   ...       ...
422414  Surgeons to remove 4-year-old's rib to rebuild...    Health
422415  Boy to have surgery on esophagus after battery...    Health
422416  Child who swallowed battery to have reconstruc...    Health
422417  Phoenix boy undergoes surgery to repair throat...    Health
422418  Phoenix boy undergoes surgery to repair throat...    Health

[422419 rows x 2 columns]


In [12]:
# Training the model with 5% of the subset just to make sure the model can train

df = df.sample(frac=0.05, random_state=1)

df = df.reset_index(drop=True)

df


,TITLE,CATEGORY
0,Murdoch's bid for Time Warner rejected,Business
1,Rescuers close in on 3 trapped Honduran miners...,Business
2,Johnny Depp - Johnny Depp Served With Legal Pa...,Entertainment
3,"Apple prepping move into ""smart home"" connecti...",Science
4,Ripped First Look: Dwayne Johnson as Brett Rat...,Entertainment
...,...,...
21116,"Fed Beige Book: Activity, labor markets improv...",Business
21117,National Agriculture Day,Business
21118,Placenta Home to Diverse Bacteria That May Aff...,Health
21119,US TV network Fox to air live 'Grease' musical...,Entertainment


In [8]:
!pwd

/root
